# Singapore
## Prepare data for node and links

In [1]:
from urllib2 import urlopen
from zipfile import ZipFile
from StringIO import StringIO
import shapefile
import geopandas as gpd
from shapely.geometry import shape  
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline

Source of file containing station locations:
https://github.com/xkjyeah/singapore-postal-codes/blob/master/mrt_stations.json

In [2]:
df = pd.read_json('mrt_stations.json')
df.head()

,Possible Locations,Station,Station Name
0,"[{u'BUILDING': u'JURONG EAST MRT STATION', u'R...",NS1,Jurong East
1,"[{u'BUILDING': u'BUKIT BATOK MRT STATION', u'R...",NS2,Bukit Batok
2,"[{u'BUILDING': u'BUKIT GOMBAK MRT STATION', u'...",NS3,Bukit Gombak
3,"[{u'BUILDING': u'CHOA CHU KANG MRT STATION', u...",NS4,Choa Chu Kang
4,"[{u'BUILDING': u'YEW TEE MRT STATION', u'ROAD_...",NS5,Yew Tee


In [3]:
import json

with open("mrt_stations.json") as json_file:
    json_data = json.load(json_file)
    

In [4]:
json_data[0]

{u'Possible Locations': [{u'ADDRESS': u'10 JURONG EAST STREET 12 JURONG EAST MRT STATION SINGAPORE 609690',
   u'BLK_NO': u'10',
   u'BUILDING': u'JURONG EAST MRT STATION',
   u'LATITUDE': u'1.33315261987297',
   u'LONGITUDE': u'103.742286544006',
   u'LONGTITUDE': u'103.742286544006',
   u'POSTAL': u'609690',
   u'ROAD_NAME': u'JURONG EAST STREET 12',
   u'SEARCHVAL': u'JURONG EAST MRT STATION',
   u'X': u'17869.0806001499',
   u'Y': u'35038.9471976683'}],
 u'Station': u'NS1',
 u'Station Name': u'Jurong East'}

In [5]:
lat = []
for i in range(len(json_data)):
    lat.append(json_data[i]['Possible Locations'][0]['LATITUDE'])
lat = [float(i) for i in lat]

In [6]:
lon = []
for i in range(len(json_data)):
    lon.append(json_data[i]['Possible Locations'][0]['LONGITUDE'])
lon = [float(i) for i in lon]

In [7]:
df['Latitude'] = lat
df['Longitude'] = lon
df = df.drop(['Possible Locations'], axis=1)

In [8]:
df.head()

,Station,Station Name,Latitude,Longitude
0,NS1,Jurong East,1.333153,103.742287
1,NS2,Bukit Batok,1.349034,103.749567
2,NS3,Bukit Gombak,1.358612,103.751791
3,NS4,Choa Chu Kang,1.385363,103.744371
4,NS5,Yew Tee,1.397535,103.747405


In [24]:
df.to_csv('master-node.csv')

In [32]:
#subnodes
df.shape

(141, 4)

In [37]:
subnode = pd.read_csv('sub-node.csv')
subnode.head()

,Unnamed: 0,Station,Station Name,Latitude,Longitude,node_id
0,8,NS10,Admiralty,1.440585,103.800988,NaN
1,34,EW9,Aljunied,1.316433,103.882906,NaN
2,13,NS16,Ang Mo Kio,1.369933,103.849558,NaN
3,88,CC12,Bartley,1.342501,103.880178,NaN
4,105,CE1,Bayfront,1.281874,103.859080,CE1_DT


In [40]:
subnode['Direction'] = subnode['Station'].astype(str)
subnode['Direction'] = subnode['Direction'].str[0:2]

In [41]:
subnode.head()

,Unnamed: 0,Station,Station Name,Latitude,Longitude,node_id,Direction
0,8,NS10,Admiralty,1.440585,103.800988,NaN,NS
1,34,EW9,Aljunied,1.316433,103.882906,NaN,EW
2,13,NS16,Ang Mo Kio,1.369933,103.849558,NaN,NS
3,88,CC12,Bartley,1.342501,103.880178,NaN,CC
4,105,CE1,Bayfront,1.281874,103.859080,CE1_DT,CE


In [43]:
subnode_transfer = subnode.dropna()
subnode_transfer.head()

,Unnamed: 0,Station,Station Name,Latitude,Longitude,node_id,Direction
4,105,CE1,Bayfront,1.281874,103.859080,CE1_DT,CE
5,121,DT16,Bayfront,1.281874,103.859080,DT16_CE,DT
12,14,NS17,Bishan,1.350839,103.848144,NS17_CC,NS
13,91,CC15,Bishan,1.350839,103.848144,CC15_NS,CC
16,94,CC19,Botanic Gardens,1.322114,103.814984,CC19_DT,CC


In [44]:
subnode['node_id'] = subnode['Station'] + '_' + subnode['Direction']
subnode.head()

,Unnamed: 0,Station,Station Name,Latitude,Longitude,node_id,Direction
0,8,NS10,Admiralty,1.440585,103.800988,NS10_NS,NS
1,34,EW9,Aljunied,1.316433,103.882906,EW9_EW,EW
2,13,NS16,Ang Mo Kio,1.369933,103.849558,NS16_NS,NS
3,88,CC12,Bartley,1.342501,103.880178,CC12_CC,CC
4,105,CE1,Bayfront,1.281874,103.859080,CE1_CE,CE


In [45]:
subnode.shape

(144, 7)

In [46]:
subnode_transfer.shape

(46, 7)

In [48]:
#get all subnodes
all_subnodes = subnode.append(subnode_transfer, ignore_index=True)

In [53]:
all_subnodes.drop('Unnamed: 0', axis=1, inplace=True)
all_subnodes.head()

,Station,Station Name,Latitude,Longitude,node_id,Direction
0,NS10,Admiralty,1.440585,103.800988,NS10_NS,NS
1,EW9,Aljunied,1.316433,103.882906,EW9_EW,EW
2,NS16,Ang Mo Kio,1.369933,103.849558,NS16_NS,NS
3,CC12,Bartley,1.342501,103.880178,CC12_CC,CC
4,CE1,Bayfront,1.281874,103.859080,CE1_CE,CE


In [55]:
all_subnodes.shape

(190, 6)

In [56]:
all_subnodes.to_csv('all-sub-nodes.csv')

In [58]:
subnode_transfer.to_csv('transfer.csv')

In [60]:
pd.read_csv('transfer_time.csv')

,Unnamed: 0,Unnamed: 0.1,Station,Station Name,Latitude,Longitude,node_id,Direction,Change Platform
0,4,105,CE1,Bayfront,1.281874,103.859080,CE1_DT,CE,1
1,5,121,DT16,Bayfront,1.281874,103.859080,DT16_CE,DT,1
2,12,14,NS17,Bishan,1.350839,103.848144,NS17_CC,NS,2
3,13,91,CC15,Bishan,1.350839,103.848144,CC15_NS,CC,2
4,16,94,CC19,Botanic Gardens,1.322114,103.814984,CC19_DT,CC,3
5,17,114,DT9,Botanic Gardens,1.322114,103.814984,DT9_CC,DT,3
6,21,37,EW12,Bugis,1.300465,103.855706,EW12_DT,EW,1
7,22,119,DT14,Bugis,1.300465,103.855706,DT14_EW,DT,1
8,26,46,EW21,Buona Vista,1.307224,103.790254,EW21_CC,EW,2
9,27,97,CC22,Buona Vista,1.307224,103.790254,CC22_EW,CC,2
